## Creating a Champions Puuid & SummonerLevel Dataset from Nothing

Since we cannot extract multiple Puuids from current riot apis, we need to cross reference apis in an iterative process to try and create a mass Puuid dataset so we can perform analytics on all of these players.

Cross reference goes as follows:
1. Find featured games and get participants info (50 players in the games)
2. Find recent matches that featured players have been in (20 matches per player)
3. Find participants from recent matches from featured participants (10 per game)

This results in ~10,000 participant dataset, however there will be double referencing and players playing together across multiple games, so we are double counting and expecting a dataset of Champions slightly under 5,000.

## Importing Packages and Setting API

Customary first steps

In [375]:
#pip install riotwatcher
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os 

# golbal variables
api_key = 'RGAPI-b592f71c-f0bc-4870-adfa-d08aed124f70'
lol_watcher = LolWatcher(api_key)
my_region = 'EUW1'

me = lol_watcher.summoner.by_puuid(my_region, 'HnublqKGARVGUs-E85zk8RjOQbnis_y0ht2LFP7skypFQ3Kb5nEsEGqZo_ZqVWxXUIFGVb8YJwyfsg')
print(me)




{'id': '2yXi7JpQoOKpvI3wkae-mkHQmlPrTBctpk82t_iBAvWZyAaZdFFWPBT8ZQ', 'accountId': '2D57NMH5W4-tSntjqPvW-AyLlKCITx7AfJhMmWZfP8vDJfz_G86YaqVK', 'puuid': 'HnublqKGARVGUs-E85zk8RjOQbnis_y0ht2LFP7skypFQ3Kb5nEsEGqZo_ZqVWxXUIFGVb8YJwyfsg', 'name': 'Nickel5870', 'profileIconId': 907, 'revisionDate': 1655316248000, 'summonerLevel': 5}


In [339]:
my_matches = lol_watcher.match.matchlist_by_puuid(my_region, me['puuid'])
#print(my_matches)

for i in my_matches:
    match_data = lol_watcher.match.by_id(my_region, i )
    
    

    #print(match_data)
    #print(match_data['metadata']['matchId'])

KeyboardInterrupt: 

## Finding Featured Games and Getting Participant Data

First we need to find featured games, this is randomly generated by the api, we then find all of the participants in these 5 featured games.

In [359]:
#getting data of multiple players in featured games

featured_games = lol_watcher.spectator.featured_games(my_region)
#print(featured_games)


#run for loop to get data for the match and the participants details
participants = []



#iterate through data
for row in featured_games['gameList']:
    players_row = {}
    #players_list_row_intermediate =featured_games['gameList']['participants'][row_1]
    #print(row)
    #print('\n heyyoooo')
    
    k=0
    for row_2 in row['participants']:
        #print(row_2)
        
        for row_3 in row_2:
            if(row_3 == 'summonerName'):
                #print(row_2[row_3],'\n heo summoner')marks.get('Physics')
                players_list_row_intermediate = row_2.get(row_3)
                print(players_list_row_intermediate)
                me = lol_watcher.summoner.by_name(my_region, players_list_row_intermediate)
                ranking = me['summonerLevel']
                puuid = me['puuid']
                #print(players_list_row_intermediate, "Ranking:", ranking)
                
                #players_row[k] = players_list_row_intermediate
                #players_row[k+1] = ranking
                my_list = [players_list_row_intermediate,puuid, ranking,]
                participants.append(my_list) 

participants
participants_df = pd.DataFrame(participants)
print(participants_df)


    

ARAM GOLEM
Not Zoom
Gniax91
Wout
Epixah
Alduin 25
Jagweuh
Wool Tea
Dainairos
EverythingIsOkay
Fking To4st Man
Turkish Airline1
TwTv Hawkella
FOX AK4SHI
omnishambles
Kite Machine
LDLC Eika
Multirole Talent
FINNNNNNN
THiS iS LOOSE
Alcadata
Pianoo
Z θ D ł λ C
AnToooDx
felipejia
Mogigo
Überdude
Justiceritoisbad
Willy GR
SafranSiggi
Xepec
Firefly2210
Sz Rexy
HU TA0 ENJOYER
XoYo0
Moi C Rono
kiLLa Laharl
xXBlackeXx
Louis Armstrong
LaChèvreCosmique
blue mean eyes
Cpt Fappy
Hardsell
Karulosu
SmookeHaze
TonkHd
Madisοn Beer
Santerama
Chains boyo
Hanva
                   0                                                  1    2
0         ARAM GOLEM  aWBlnvq2HpzOSPAZlfd4J88AjgN9uZfA6HIy13QoNgjbhF...  355
1           Not Zoom  -gnqSv5UA9xQSLrqUFn_kzmX5A9YbzTkpke_MME4QbOpgs...  736
2            Gniax91  J-CR7gCVQf2cR3Lbe0D2ybElDlZSgYYDSrAZ11ml3_20sy...  363
3               Wout  kahOInpbCgYEL2-9hEt1axAxrCuf5zgOP_pRL1mdPcV6YM...  252
4             Epixah  PxwZujw8XR6z9grvS7309LjqZPhjqJz37k4l15IpT5udhJ

Create a dataframe of these featured players and sort by rank (not necessary)


In [360]:
#putting columns names
columnNames = ['summonerName','puuid','summonerRanking']
participants_df.columns = columnNames


print(participants_df)



        summonerName                                              puuid  \
0         ARAM GOLEM  aWBlnvq2HpzOSPAZlfd4J88AjgN9uZfA6HIy13QoNgjbhF...   
1           Not Zoom  -gnqSv5UA9xQSLrqUFn_kzmX5A9YbzTkpke_MME4QbOpgs...   
2            Gniax91  J-CR7gCVQf2cR3Lbe0D2ybElDlZSgYYDSrAZ11ml3_20sy...   
3               Wout  kahOInpbCgYEL2-9hEt1axAxrCuf5zgOP_pRL1mdPcV6YM...   
4             Epixah  PxwZujw8XR6z9grvS7309LjqZPhjqJz37k4l15IpT5udhJ...   
5          Alduin 25  WkO51bmDhnwnhSgpyKAQcMPn5erffN6aRYlJXzuUzok-Yr...   
6            Jagweuh  jbGcSydOulmJ_Jdy-YF3NqZbORsSHrsfW8JeGB1Quj_nql...   
7           Wool Tea  xM_IwTbmTiWUSN7nGLSwpVI1SU7uWPMrBmjSYtj__wp5Vp...   
8          Dainairos  qDhuPwLPpHx4ryoOC7oH4zoSZbbspB8SUJB3wiHbZ3sRx2...   
9   EverythingIsOkay  D6TgmksDVhhVEuLAkKBQUwvdNobe3u5RWBVx5T5MFSV4ys...   
10   Fking To4st Man  7C_w5ZaT644dxmJtdEEaoYnff8Lo2gXmHvKUOJwnqI9HvE...   
11  Turkish Airline1  m5AbiLmme4h_o489bko8Jmk6PtlJ4lFr53HE13E92MXvwT...   
12     TwTv Hawkella  DUm

In [361]:

participants_df.sort_values('summonerRanking')




,summonerName,puuid,summonerRanking
12,TwTv Hawkella,DUmbiMQLrUQ9THBMvWlXlfCY504kjAJT_3XiPRGGijspAx...,60
11,Turkish Airline1,m5AbiLmme4h_o489bko8Jmk6PtlJ4lFr53HE13E92MXvwT...,95
39,LaChèvreCosmique,4XI9gi_sBWJJMtOdSy8sS97VuLopWKi6HUkEXoDNdcRsue...,109
5,Alduin 25,WkO51bmDhnwnhSgpyKAQcMPn5erffN6aRYlJXzuUzok-Yr...,136
23,AnToooDx,_VtHexwyzyVhjUzpEVGegGztx86RTunODuJikmXgfD_iE3...,150
6,Jagweuh,jbGcSydOulmJ_Jdy-YF3NqZbORsSHrsfW8JeGB1Quj_nql...,162
17,Multirole Talent,1GpHSrz1ECjM4rhnyorY4s_IcYXjMGVYJCGdWo_hGwoPNT...,167
19,THiS iS LOOSE,GfBnpGY34sjYn5C3iGfj2TFv3Di5BxC5wZSYwJ8dIuPv04...,189
13,FOX AK4SHI,xT_HFYhm8vl33-Yvy8j9PG2-O6nm-Sj9FXo6hurGyCLK93...,195
26,Überdude,ntdgR4hzO8XxvNMGJACqaQYFzRTELKCimID7U5yMLHNfc2...,250


In [362]:
#saving the data to a folder location
os.makedirs('folder/users', exist_ok=True)  
participants_df.to_csv('folder/users/featured_game_summoners_3.csv')  

## Getting all the Recent Matches that Featured Players have Played

Now to expand our Champions dataset, we will cross reference all of the matches featured players have been in. This will give us a further list of champions that they have played against to expand our current champions data set.

Iterate through matches, and get rid of any duplicate matches as some of these players will have played in the same games.

In [363]:
#get all. match ids of the players
matches_list = []
for iterate_through_featured_game_participants in participants:
    unique_puiid = iterate_through_featured_game_participants[1]
    my_matches = lol_watcher.match.matchlist_by_puuid(my_region, unique_puiid)
    #print(my_matches)
    
    for j in my_matches:
        my_match_list = [j]
        matches_list.append(my_match_list)
        
matches_list

[['EUW1_5927611285'],
 ['EUW1_5927539462'],
 ['EUW1_5927518171'],
 ['EUW1_5927596195'],
 ['EUW1_5927585326'],
 ['EUW1_5926483168'],
 ['EUW1_5926377495'],
 ['EUW1_5926405090'],
 ['EUW1_5926422194'],
 ['EUW1_5926280773'],
 ['EUW1_5926236766'],
 ['EUW1_5925906342'],
 ['EUW1_5925864078'],
 ['EUW1_5925375163'],
 ['EUW1_5925360999'],
 ['EUW1_5925303568'],
 ['EUW1_5925250626'],
 ['EUW1_5924987390'],
 ['EUW1_5924924122'],
 ['EUW1_5924891616'],
 ['EUW1_5927658165'],
 ['EUW1_5927606130'],
 ['EUW1_5927602573'],
 ['EUW1_5927510146'],
 ['EUW1_5927547844'],
 ['EUW1_5927596195'],
 ['EUW1_5927594777'],
 ['EUW1_5927593939'],
 ['EUW1_5927385503'],
 ['EUW1_5927364268'],
 ['EUW1_5927343540'],
 ['EUW1_5925811350'],
 ['EUW1_5925740437'],
 ['EUW1_5925729857'],
 ['EUW1_5925708942'],
 ['EUW1_5925648005'],
 ['EUW1_5924113083'],
 ['EUW1_5924132352'],
 ['EUW1_5924131707'],
 ['EUW1_5924111122'],
 ['EUW1_5927638798'],
 ['EUW1_5926534843'],
 ['EUW1_5926531421'],
 ['EUW1_5926399382'],
 ['EUW1_5926376224'],
 ['EUW1_59

In [364]:
#get rid of duplicate matches and convert to df.

matches_list_n_duplicates = [i for n, i in enumerate(matches_list) if i not in matches_list[:n]] 


matches_list_df = pd.DataFrame(matches_list_n_duplicates)

#putting columns names
columnNames_matches = ['EUW1_match']
matches_list_df.columns = columnNames_matches


print(matches_list_df)

          EUW1_match
0    EUW1_5927611285
1    EUW1_5927539462
2    EUW1_5927518171
3    EUW1_5927596195
4    EUW1_5927585326
..               ...
929  EUW1_5919227728
930  EUW1_5919130138
931  EUW1_5919097602
932  EUW1_5919066254
933  EUW1_5918967939

[934 rows x 1 columns]


In [365]:
## confirm no duplicates
matches_list_df.groupby('EUW1_match').EUW1_match.nunique()

EUW1_match
EUW1_5801718550    1
EUW1_5801771842    1
EUW1_5802658009    1
EUW1_5802690267    1
EUW1_5802745446    1
                  ..
EUW1_5927686639    1
EUW1_5927688385    1
EUW1_5927694250    1
EUW1_5927694625    1
EUW1_5927696576    1
Name: EUW1_match, Length: 934, dtype: int64

In [366]:
#matches list to csv
matches_list_df.to_csv('folder/users/all_matches_featured_participants_3.csv') 

## Finding a List of Participants Based off of Featured Games at Current Date/Time

Next we want to extract even more participants from the game so we can increase our Champions puuid dataset. We can do this by cross referencing all the participants in the matches that was stored in the previous set of codes.

We iterate through and throw a try and exception as server sometimes has errors.

We also need to get rid of duplicate players, since there is the potential for duplicate players across these games.

Finally, the new participant dataset is exported and turned into a df for further analysis later.

In [367]:
#now lets get all of the puuids of each player in these matches
large_participant_list = []

i = 0
for iterate_through_matches in matches_list:
    #need to convert list to string, so the api can accept the match ids correctly.
    list_to_string_intermediate_match_id = ''.join(iterate_through_matches)
    
    #need to add a sleep time so we do not inundate server and exceed our request limit
    time.sleep(0.01)
    #print(list_to_string_intermediate_match_id)
    i = i+1
    print(i)
    #call the api to get match info, need to have exception for HTTPError to skip data
    try:
        match_info = lol_watcher.match.by_id(my_region,list_to_string_intermediate_match_id )
        #do calls to acquire all the participants ids in the match and make a list.
        for participantsInUniqueMatch in match_info['metadata']['participants']:
            my_participant_list = [participantsInUniqueMatch]
            large_participant_list.append(my_participant_list) 
            
    except ApiError as err:
        if err.response.status_code == 503:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        else:
            continue

    
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [368]:
#print(large_participant_list)

#get rid of duplicates
large_participants_dup = [i for n, i in enumerate(large_participant_list) if i not in large_participant_list[:n]] 

large_participants_df = pd.DataFrame(large_participants_dup)


#putting columns names
columnNames = ['puuid']
large_participants_df.columns = columnNames

print(large_participants_df.size)
print(large_participants_df)



5519
                                                  puuid
0     zRp0xtj43YT9qLeWDfk0JXQaLpqcKJ_oEpIxoG0ZR7AU5j...
1     dIqO-wlwQQzLGsAgKLj4xKftKMAo-pyt_4hZAgPofmnKUJ...
2     6paO3ZoK2HcOZwI4ziJi8OAP1LzvLrGIeAyVRBUUSrkHC6...
3     LMruNOMBPXineVVRuhLAOkYOtaHR2LhF4yO-OVKYKPRYsM...
4     bIMG1gzIGu0fcDXIU0LlBSp-L9rezRf5tu5PVQs8qLG27n...
...                                                 ...
5514  hiL9s1gwwI1JEF1zlb33uposW6yqBPNNlqUI_AhTyQGlV4...
5515  cSUUIYXCUsXrXQYb6iO6TFFihUvoRqttpqJGfHpCLIf5i-...
5516  yPc43_r0QxsARWPqRGjpFs9M6WLf9G371tVqpMXjjN7VyH...
5517  GHB3Keq3JPbL5iZDZehdB7u45n48ptPLUHYp8RwX2ghuKX...
5518  FYEJA7KhJ6iuDwXFpmUxmcrnexvgXSSXzy3miNwR1BKhcb...

[5519 rows x 1 columns]


In [369]:
#big participants puuid data list to csv
large_participants_df.to_csv('folder/users/lots_of_participants_3.csv') 

## Finding the Champions Rankings 

Next we want to find the rankings of all of the champions to try and fine the lowest ranked ones and determine engagement.

The puuid summoner API is used, and there is a try and except used in case that the server fails.

Additionally the data is put into a dataframe and sorted and saved to more easily show which champions are new to the game

In [370]:
#find the ranking
large_participant_ranking = []
j =0
for i in large_participant_list:
    #j=j+1
        
    list_to_string_intermediate_participant = ''.join(i)
    try:
        me = lol_watcher.summoner.by_puuid(my_region, list_to_string_intermediate_participant)
        puuid = me['puuid']
        name = me['name']
        ranking = me['summonerLevel']
        list_partic_rank = [name,ranking, puuid]
        large_participant_ranking.append(list_partic_rank)
            
    except ApiError as err:
        if err.response.status_code == 503:
            #print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            continue
        else:
            continue
    
    
    #print(ranking)

KeyboardInterrupt: 

In [373]:
large_participant_ranking

large_participant_ranking_df = pd.DataFrame(large_participant_ranking)


columnNames = ['summonerName','summonerRanking','puuid']
large_participant_ranking_df.columns = columnNames

large_participant_ranking_df.sort_values('summonerRanking')

,summonerName,summonerRanking,puuid
1474,Nickel5870,5,HnublqKGARVGUs-E85zk8RjOQbnis_y0ht2LFP7skypFQ3...
2122,Ripanele,12,_NiDmjH75tQhk1InNBy_TVbZi8pJgvnD_bsHvVJM91GptQ...
2132,Ripanele,12,_NiDmjH75tQhk1InNBy_TVbZi8pJgvnD_bsHvVJM91GptQ...
2086,Mvou,19,5tCG68uMo8iU3QEbV1l4FFfCfvohagOqlXxoqtC_qkVtp8...
2076,Mvou,19,5tCG68uMo8iU3QEbV1l4FFfCfvohagOqlXxoqtC_qkVtp8...
...,...,...,...
1,HûaHuâ,1201,dIqO-wlwQQzLGsAgKLj4xKftKMAo-pyt_4hZAgPofmnKUJ...
370,Kitty Nicole,1216,ePO-fVMyQFdvCGjzIdDtJ_xLPSq2mL5gEGhRo4PvU8Xur3...
2250,Qnoxs,1348,wSy7L6SkmKwod9Jsur2XXPycEywOffupEnpj69rjNWfYBs...
2934,Qnoxs,1348,wSy7L6SkmKwod9Jsur2XXPycEywOffupEnpj69rjNWfYBs...


In [374]:
#big participants with rankings names and puiid data list to csv
large_participant_ranking_df.to_csv('folder/users/rankings_of_participants_3.csv') 

In [372]:
large_participant_ranking_df.groupby('summonerName').summonerRanking.nunique()

summonerName
 Équinöx            1
0101110101011       1
0zymandias          1
1 VENATOR           1
12 yo cute egirl    1
                   ..
Τurtlε              1
γordle              1
θΘθΘθ               1
ρoro                1
τεsτγcφφł           1
Name: summonerRanking, Length: 1795, dtype: int64

In [355]:
large_participant_ranking_df.groupby('puuid').puuid.nunique()

puuid
-08c1OFwv7dPRVRGJJS_ZyNZdzH-OArztSiEBmxUmfW0NeyuEYJTa4anE8IlrjFTvEI5X8kM71lw-g    1
-0JdcYpRwJA9Z6XMT2NgWFEpstE9zrJigCEPoqB1KR-VRodXN-jCyqRMcGgtRE_ms0M1LrNgnsaEtg    1
-303p3c8heA8dbLhZIm3-sDshGLQliBlLHQQf-G7XHbNR-rNcBDlVfcIdWHPOQFUWoP_tJ-Lk-pVMw    1
-6qvFqt2SSZ_DVIChiZeNrjY-d4M3MjC6H_zpTGZxCOCaw-vTtMG41eZ8RQAZtQT_e74yq69SBdCZQ    1
-8nKPYaPo976tl2WdRBU2wFFWw-rB6JZmNIyTdiieLkSTur0N6y9L_7SryerZ-_Ru6BJXStb91KmzA    1
                                                                                 ..
znJwLqjgnaCjD8R8VWTVMy_H3tvsbB4suUqKfb3mMAm9VS-IQSXsQbcPw55SNC2-a4do_1JiMTnGkQ    1
zpME0JJUiIdXTz6bgXSfpU8-dnEknvcTrPU_M5AnIn8w9HiYk-6GSZZZLgYLBNmlohi-JsCUor9oDg    1
zu5hh7w3dSCL-_Di8727LUhgKdpHWkV40JkOvFlsZrJMwJNeV7tlr5Qyt7cNYC2IwKCQeslfSogI0w    1
zujzMK4wqZA1822kIWW8oBDZf_45p9aLLQd5o1ARP3pwcRFTX5jezJySQstwwhynk2ezCf9A_ke6SA    1
zybuYlYYxljy0gqqSYM0HlP5rmyx_rSuIVb3CJXjFLkXgLLyQr53MadPfbTJ7x4kdgowsmRKS9zISA    1
Name: puuid, Length: 1795, dtype: int64

In [356]:

large_participant_ranking

[['ECHENIKE4x4',
  426,
  'fcltQPG7yCwr4rpFov9C18FjA_IcPQrk8Ra6xnRQC9mlLfg64umywzp2Io_z-JgWVfdtuqbJLJoq4g'],
 ['lucmar',
  391,
  'uUyKnSXxGlrK4lBmsc9aphnfNdX9Y3sbszu7XiHS4220BpJ95VRdfnoWyrE-tva9X6bAucbhrwLf1w'],
 ['RickyRick',
  298,
  'fmQ-ShBIJ0z2A2dIFAB8-Z2c7GoPQ5fVn7n4EEKVLPZlsWxDOjkc0YhVu6KPqjD1lEISY89_8qrSlQ'],
 ['BREEEUUUUHHHHHH',
  536,
  'Ty-tOx4ru4nRMZyLtCKH7KgqoT4_Ej4-X-0NM-hFquKHc1zWHKg6R5sy6RAQo8XJ8dykA-hFbrjMBQ'],
 ['Αscended',
  256,
  'Ippd3H4PZpdVaAYEHwbsBwHcMCxdrwUWZ5hGdTCSc_rwmqcdAgE1uT8FkvlYqSGdA57kg3OMcJw4zw'],
 ['Kn1ghtly',
  173,
  'tPgj8KJQlPiGtTcmhR91pj-gdJ-bPHmd84FhLDEOhec8BPIFldlV14Cfb-NTSaEb3SQGbCMWWnHaIg'],
 ['KaiteZuMirDuHund',
  376,
  'mBDO0Backikq4a1ZCTMC2HyFt8iONZzpZf51ln-4-e3nFdt8IlObC4Iv9fITzu0rQYBdjFmC0mV9zA'],
 ['mightymammut',
  534,
  'CG7JGiaIT2JgJLrc1NCGonyvubtiu1VuhRnIZix7Hnw9y68nqrpZraZE-5eVUAq3doZRpgED4Zgkcw'],
 ['sYNCCRO',
  196,
  'olMOGd4Xn0lN3gDQfWv_7FvwAUaedugL532jkZZtSi0c4xe3TUlfK1ixFMt-K3279Xc3JqDN3kM_Vw'],
 ['Rakhog',
  321,
  'JkL2